# 요구사항 1 : tatanic_dataset.py 분석

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X) # x -> FloatTensor로 변환
    self.y = torch.LongTensor(y) # y -> LongTensor로 변환 (target)

  def __len__(self): # 데이터셋의 크기 반환
    return len(self.X) 

  def __getitem__(self, idx): # 데이터셋에서 특정 인덱스 데이터를 가져옴
    feature = self.X[idx] 
    target = self.y[idx]
    return {'input': feature, 'target': target} # 딕셔너리 형태로 반환

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape # 데이터셋 크기, 입력 데이터, 레이블 크기 출
    )
    return str

In [3]:
class TitanicTestDataset(Dataset): # 테스트용 데이터셋
  def __init__(self, X):
    self.X = torch.FloatTensor(X) # 입력 데이터를 FloatTensor로 변환

  def __len__(self): # 데이터셋 크기 반환
    return len(self.X)

  def __getitem__(self, idx): 
    feature = self.X[idx]
    return {'input': feature} # 입력 데이터를 딕셔너리 형태로 반환

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape # 데이터셋 크기 및 입력 데이터 모양 출
    )
    return str

In [4]:
def get_preprocessed_dataset():
    # cvs의 파일 경로를 지정하기 위하여 수정함
    # 주피터 노트북 경로 수정
    train_data_path = os.path.join(os.getcwd(), "train.csv")
    test_data_path = os.path.join(os.getcwd(), "test.csv")

    train_df = pd.read_csv(train_data_path) # 학습 데이터 로드
    test_df = pd.read_csv(test_data_path) # 테스트 데이터 로드

    all_df = pd.concat([train_df, test_df], sort=False) # 학습과 테스트 데이터를 결과

    #전처리 6단계 적용
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # 학습 데이터 추출
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    # 레이블 추출
    train_y = train_df["Survived"]
    # 학습 데이터 추출
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    # TitanicDataset 객체 생성
    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset) # 80:20 비율로 학습 및 검증 데이터셋 분할
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    # TitanicTestDataset 객체 생성
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

In [5]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df



In [6]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df
    

In [7]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df



In [8]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df



In [9]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df



In [10]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df



In [11]:
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, 30), # 인풋 레이어 사용
      nn.ReLU(), # 활성화 함수 
      nn.Linear(30, 30), # hidden 레이어
      nn.ReLU(), # 활성화 함수
      nn.Linear(30, n_output), # 출력 레이어
    )

  def forward(self, x):# 순전파 행
    x = self.model(x)
    return x



In [12]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader)) # 첫 번째 배치를 가져옴
  print("{0}".format(batch['input'].shape)) # 인풋 데이터 모양 출력
  my_model = MyModel(n_input=11, n_output=2) # 모델 인스턴스 생성
  output_batch = my_model(batch['input']) # 출력 값 계산
  prediction_batch = torch.argmax(output_batch, dim=1) # 예측 값 계산
  for idx, prediction in enumerate(prediction_batch, start=892): # 인덱스 892 부터 예측값 출력
      print(idx, prediction.item())


if __name__ == "__main__":
    # 데이터셋 전처리
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

    # 샘플 출력
  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

    #배치 단위로 데이터를 처리하는 데이터로드 생성
    # 배치 사이즈는 16개, 데이터는 랜덤, 테스트 데이터 셋은 전체 데이터 크기 사용
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

C:\Users\nns50\AppData\Local\Temp\ipykernel_26840\844162164.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\nns50\AppData\Local\Temp\ipykernel_26840\1033655538.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 43.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
1 - tensor([ 2.0000,  1.0000, 66.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 1.0000,  1.0000, 45.0000,  0.0000,  0.0000, 26.5500,  2.0000, 87.5090,
         4.0000,  0.0000,  1.0000]): 0
3 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000, 56.4958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
4 - tensor([ 1.0000,  1.0000, 60.0000,  1.0000,  1.0000, 79.2000,  0.0000, 87.5090,
         2.0000,  2.0000,  0.0000]): 1
5 - tensor([ 3.0000,  0.0000, 22.0000,  1.0000,  0.0000, 15.5000,  1.0000, 13.3029,
         1.0000,  1.0000,  0.0000]): 1
6 - tensor([ 1.0000,  0.0000, 18.0000,  0.0000,  2.0000, 79.6500,  2.0000, 87.5090,
         1.0000,  2.0000,  0.0000]): 1
7 - tensor([ 3.00

# 요구사항 2 : titanic 딥러닝 모델 훈련 코드 및 Activation Function 변경
- MyModel 에서 활성 함수를 계속 변경함

# titanic_wandb.py

In [13]:

import argparse
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import torch
from torch.utils.data import DataLoader

import wandb
from torch import nn, optim

# 주피터 노트북은 __Path(__file__)이 안됨
# BASE_PATH = str(Path(__file__).resolve().parent.parent.parent) # BASE_PATH: /Users/yhhan/git/link_dl
BASE_PATH = str(Path(os.getcwd()).parent) # 기본 PATH 설정
print(BASE_PATH, "!!!!!!!")

C:\Users\nns50\git\link_dl\_02_homeworks !!!!!!!


In [14]:
import sys
sys.path.append(BASE_PATH)

CURRENT_FILE_PATH = os.getcwd() # 주피터 노트북은 __Path(__file__)이 안됨
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
if not os.path.isdir(CHECKPOINT_FILE_PATH): os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))
print(CURRENT_FILE_PATH, "!!!!!") # 현재 파일 주소
print(CHECKPOINT_FILE_PATH, "!!!!!") # 체크포인트 폴더 주소


C:\Users\nns50\git\link_dl\_02_homeworks\homework_2 !!!!!
C:\Users\nns50\git\link_dl\_02_homeworks\homework_2\checkpoints !!!!!


In [15]:
from titanic_dataset \
import get_preprocessed_dataset

# 각 전처리된 data_loader를 가져오는 역할
def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset() # 전처리된 dataset
  print(train_dataset)
  print("&" * 50, 1)
    # Dataset을 DataLoader로 변환
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=32, shuffle=False)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)
  return train_data_loader, validation_data_loader, test_data_loader


In [16]:
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

      # 레이어 층을 쌓은 모델
    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]), # FCN (완전 연결 네트워크)
      nn.PReLU(), # 활성 함수, ReLU, ELU, LEakyReLU, PReLU 사용을 해봄
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.PReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

In [17]:
# 모델을 최적화함 (optimizer)
def get_model_and_optimizer():
  my_model = MyModel(n_input=11, n_output=2) # 입력 파라미터 11개, 이진 분류 => 출력 파라미터 2개
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate) # SGD 최적화

  return my_model, optimizer



# 요구사항 3 : 테스트 및 submission.csv

- validation_loss가 가장 낮은 지점을 저장함
- 이후 저장한 지점을 활용하여 테스트를 실행하며 실행 결과를 저장함

In [18]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs # 학습 횟수
  loss_fn = nn.CrossEntropyLoss() # 손실 함수 Cross Entropy Loss 사용
  best_val_loss = float('inf') # 검증 손실 최솟값 저장, 최적 모델 저장에 이용
  next_print_epoch = 100 # 학습 진행 상태 출력 주기

# 전체 학습 데이터셋을 처리
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
      # 훈련 단계
    model.train()
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target']
      output_train = model(input) # 모델에 입력
      loss = loss_fn(output_train, target) # 예측값 - 실제값 손실 계산
      loss_train += loss.item() 
      num_trains += 1

      optimizer.zero_grad() # 기울기를 초기화 (누적 방지)
      loss.backward() # 역전파 계산 
      optimizer.step() # 파라미터 업데이트

      # 검증 단계
    model.eval()
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad(): # 검증 단계에서 기울기 계산 멈춤
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target']
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item() # 검증 loss 계산
        num_validations += 1
      # 검증 loss 최저 기록 갱신 및 저장
    val_loss_avg = loss_validation / num_validations
    if val_loss_avg < best_val_loss:
      best_val_loss = val_loss_avg
      save_checkpoint(model, epoch, os.path.join(CHECKPOINT_FILE_PATH, "best_model.pth"))

      # 에포크마다 학습 및 검증 loss
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": val_loss_avg
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

In [19]:
# 모델의 가중치와 epoch를 저장하여 테스트에 사용
def save_checkpoint(model, epoch, path):
  torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
  }, path)

In [20]:
def test_model_and_create_submission(model, test_data_loader):  # 최적 모델을 사용한 테스트 함수
  best_model_path = os.path.join(CHECKPOINT_FILE_PATH, "best_model.pth") 
  checkpoint = torch.load(best_model_path)  # 최적 모델 로드
  model.load_state_dict(checkpoint['model_state_dict'])  
  print(f"Loaded best model from epoch {checkpoint['epoch']} for testing.")  
  create_submission(model, test_data_loader)  # 테스트 결과로 제출 생성


In [21]:
def create_submission(model, test_data_loader):
  model.eval()
  predictions = []
  with torch.no_grad():
    for test_batch in test_data_loader:
      input = test_batch['input']
      output = model(input)

      # 디버깅: 출력 차원 확인
      print(f"Model output shape: {output.shape}")

      prediction = torch.argmax(output, dim=1)
      predictions.extend(prediction.tolist())



  # submission.csv 생성
  submission_df = pd.DataFrame({
    'PassengerId': range(892, 892 + len(predictions)),
    'Survived': predictions
  })
  submission_df.to_csv('submission.csv', index=False)
  print("Submission file created: submission.csv")


In [22]:
def main(args):
    # 현재 시간 스탬프 저장
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)
  print("!" * 50, 1)

    # Data_loader 생성
  train_data_loader, validation_data_loader, test_data_loader = get_data()

    # 모델 최적화 생성
  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

    # 학습 및 검증
  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )

    # 최적 결과를 활용한 테스트 및 제출 파일 생성
  test_model_and_create_submission(linear_model, test_data_loader)

  wandb.finish()



In [23]:
# https://docs.wandb.ai/guides/track/config
# Jupyter Notebook은 기본적으로 내부에서 여러 인자를 처리하며, 이를 argparse가 제대로 처리하지 못해 발생하는 오류로 변경
# if __name__ == "__main__":
#   parser = argparse.ArgumentParser()

#   parser.add_argument(
#     "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
#   )

#   parser.add_argument(
#     "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
#   )

#   parser.add_argument(
#     "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
#   )

#   args = parser.parse_args()

#   main(args)
if __name__ == "__main__":
    class Args:
        wandb = False  # 혹은 True로 설정 가능
        batch_size = 64  # 원하는 배치 크기로 설정
        epochs = 1000  # 원하는 에폭 설정

    args = Args()  # 직접 객체 생성

    main(args)  # 기존 main 함수에 수동으로 전달


{'epochs': 1000, 'batch_size': 64, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 1
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& 1
################################################## 1
Epoch 100, Training loss 0.5658, Validation loss 0.6184
Epoch 200, Training loss 0.5560, Validation loss 0.6034
Epoch 300, Training loss 0.5514, Validation loss 0.5959
Epoch 400, Training loss 0.5474, Validation loss 0.5855
Epoch 500, Training loss 0.5316, Validation loss 0.5669
Epoch 600, Training loss 0.5117, Validation loss 0.5542
Epoch 700, Training loss 0.5128, Validation loss 0.5302
Epoch 800, Training loss 0.5109, Validation loss 0.6103
Epoch 900, Training loss 0.4882, Validation loss 0.6697
Epoch 1000, Training loss 0.4675, Validation loss 0.4682


C:\Users\nns50\AppData\Local\Temp\ipykernel_26840\256124920.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(best_model_path)  # 최적 모델 로드


Loaded best model from epoch 992 for testing.
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([32, 2])
Model output shape: torch.Size([2, 2])


# 요구사항 4 : submission.csv 제출 및 등수확인

![점수 및 위치](https://github.com/nns503/link_dl_ns/blob/main/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202024-10-25%20163417.png?raw=true)

# 숙제 후기

실제로 데이터셋을 활용하여 딥러닝 훈련을 해본 것 같아서 의미가 있는 과제였던 것 같다.
또한 타이타닉 데이터셋을 활용하여 테스트 결과를 만드는 과정이 힘든 것도 있었지만 해결을 하면서 배운 것이 많은 것 같다.
활성 함수에 따라 훈련 결과 편차가 있는 것을 보고 활성 함수 그리고, loss 함수 등을 훈련 목적에 따라 잘 선택하는 것도 중요하다고 느꼈다.